In [1]:
cd ..

e:\research\FedImpute


e:\research\FedImpute\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import numpy as np
import loguru
import sys

## Step1 Prepare the data

We simulate a simple gaussain random data. Remember to make the `data_config` dictionary to be like the following, the keys in `data_config` should be consistent with the keys used for FedImpute.

In [4]:
data = np.random.rand(10000, 10)
data_config = {
    'task_type': 'regression',
    'clf_type': None,
    'num_cols': 9,
}

## Step2 Simulate Federated Missing Data Scenario

Import simulator and simulate the scanrios, details of parameters can be found in documentation

In [5]:
%load_ext autoreload
%autoreload 2
from fedimpute.simulator import Simulator
simulator = Simulator(debug_mode=False)
simulation_results = simulator.simulate_scenario(data, data_config, num_clients = 10, verbose=1)

Data partitioning...
Missing data simulation...
Simulation done. Using summary function to check the simulation results.


In [6]:
simulation_results = simulator.simulate_scenario_lite(
    data, data_config, num_clients = 10, dp_strategy='niid-dir@0.1', ms_scenario = 'mar-heter', verbose=1
)

Data partitioning...
Missing data simulation...
Simulation done. Using summary function to check the simulation results.


## Step3 Run the Federated Imputation

Run the federated imputation with the simulated data

In [6]:
%load_ext autoreload
%autoreload 2
from fedimpute.execution_environment import FedImputeEnv

env = FedImputeEnv()
env.reset_env()
env.configuration(imputer = 'miwae', fed_strategy='fedavg', fit_mode = 'fed')
env.setup_from_simulator(simulator = simulator, verbose=1)
print(env.imputer_name, env.fed_strategy_name)

env.run_fed_imputation(run_type='parallel')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setting up clients...
Setting up server...
Setting up workflow...
Environment setup complete.
miwae fedavg
2024-07-30 18:13:11.450 | INFO     | fedimpute.execution_environment.workflows.workflow:eval_and_track_parallel:182 - 
Initial: rmse - 0.576906701804577 ws - 0.2661060649722692


Global Epoch:   0%|          | 0/150 [00:00<?, ?it/s]

epoch: 0
receiving results
aggregating
impute and update
epoch: 1
receiving results
aggregating
impute and update
epoch: 2
receiving results
aggregating
impute and update
epoch: 3
receiving results
aggregating
impute and update
epoch: 4
receiving results
aggregating
impute and update
epoch: 5
receiving results
aggregating
impute and update
epoch: 6
receiving results
aggregating
impute and update
epoch: 7
receiving results
aggregating
impute and update
epoch: 8
receiving results
aggregating
impute and update
epoch: 9
receiving results
aggregating
impute and update
epoch: 10
receiving results
aggregating
impute and update
epoch: 11
receiving results
aggregating
impute and update
epoch: 12
receiving results
aggregating
impute and update
epoch: 13
receiving results
aggregating
impute and update
epoch: 14
receiving results
aggregating
impute and update
epoch: 15
receiving results
aggregating
impute and update
epoch: 16
receiving results
aggregating
impute and update
epoch: 17
receiving resu

## Step4 Evaluate the Imputation Outcome

Evaluate the imputation outcome, you can choose which of three evaluation aspects, you want to evaluate 

In [13]:
%load_ext autoreload
%autoreload 2
from fedimpute.evaluation import Evaluator

evaluator = Evaluator()
# eval_ret = evaluator.evaluate(env, ['imp_quality', 'pred_downstream_local', 'pred_downstream_fed'])
eval_ret = evaluator.evaluate(env, ['imp_quality'])
evaluator.show_results()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Evaluating imputation quality...
Evaluation completed.
Evaluation Results
Imputation Quality             | rmse: 0.293 (0.00) nrmse: 0.682 (0.02) sliced-ws: 0.082 (0.01) 


In [14]:
np.array(evaluator.results['imp_quality']['imp_quality']['rmse']).mean()

0.2930719807545368